In [2]:
import pandas as pd
import datetime as dt1
from datetime import datetime as dt2
from os import listdir
from os.path import isfile, join
import re
import numpy as np

In [2]:
##define the range of dates to select for
#fromdate = dt1.date(1984,1,1)
#todate = dt1.date(2018,1,1)
#def comparetime(a):
#    time = dt2.strptime(a,"%Y-%M")
#    if (time.year >= fromdate.year) & (time.year <= todate.year):
#        return True
#    else:
#        return False

In [3]:
##get the list of filenames in the dir
onlyfiles = [f for f in listdir("gsoy-latest/") if isfile(join("gsoy-latest/", f))]
USindex = [string for string in onlyfiles if string.startswith("US")]


In [ ]:
### loop through files and merge the ones that are within the specified year range
a = 0
for ifile in USindex:
    df = pd.read_csv("gsoy-latest/" + ifile)
    timelim = (df['DATE']>=1984) & (df['DATE'] <= 2018)
    if np.sum(timelim) > 0:
        if a == 1:
            merged_df = pd.concat([merged_df,df[timelim==True]],sort=False)
        else: 
            a = 1
            merged_df = df[timelim==True]

In [ ]:
export_csv = merged_df.to_csv (r'merged_weather.csv', index = None, header=True) 

### merge different csv files

In [8]:
#df1 = pd.read_csv("merged_weather1.csv")
#df2 = pd.read_csv("merged_weather2.csv")
#df3  = pd.read_csv("merged_weather3.csv")

C:\Users\Qiuyan\Miniconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (39,63,65,67,69,71,73,77,79,81,83,85,87,90,92,94,96,98,100,103,105,107,109,111,113,115,117,119,121,123,125,127,129,131,133,135,137,142,153,155,161,163,165,167,173,175) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [9]:
#merged_df = pd.concat([df1,df2,df3],sort = False)
#export_csv = merged_df.to_csv (r'merged_weather.csv', index = None, header=True) 

### delete some columns
The columns with _ATTRIBUTES can be deleted -> they provide information for the corresponding field, i.e, how many days of data are missing etc...

In [16]:
colnames = list(merged_df) ## 
colname_kept = []
for colname in colnames:
    if '_ATTRIBUTES' not in colname:
        colname_kept.append(colname)
merged_df = merged_df[colname_kept]


,AWND,CDSD,CLDD,DATE,DP01,DP10,DP1X,DSND,DSNW,DT00,...,LX06,LX07,LX08,MN06,MN07,MN08,MN09,MX06,MX07,MX08
0,NaN,0.0,0.0,1987,244.0,205.0,60.0,45.0,15.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,0.0,0.0,1988,254.0,200.0,55.0,NaN,36.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,0.0,0.0,1989,187.0,164.0,56.0,123.0,25.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,0.0,0.0,1990,178.0,157.0,47.0,149.0,36.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,0.0,0.0,1991,214.0,202.0,55.0,127.0,37.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
cols_tokeep = ["NAME","DATE","STATION","TMAX","TMIN","TAVG","EMXT","EMNT","DX90","DT32","PRCP","SNOW","AWND"]

In [22]:
trimmed_df = merged_df[cols_tokeep]

In [45]:
trimmed_df['STATE'] = trimmed_df["NAME"].apply(lambda x: str(x)[-5:-3])

C:\Users\Qiuyan\Miniconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [50]:
trimmed_df = trimmed_df.drop(['NAME','STATION'],axis=1)

In [67]:
agg_df = trimmed_df.groupby(['DATE','STATE'],as_index = False).agg('mean')

In [60]:
export_csv = agg_df.to_csv (r'processed_weather.csv', index = None, header=True) 